In [1]:
# if u run this notebook in collab, install this dependences

# !pip install transformers
# !pip install datasets
# !pip install wandb

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
import numpy as np
from datasets import Dataset, load_dataset, DatasetDict
import wandb
from datetime import datetime
import pandas as pd
import datasets
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

/home/pavel/Projects/Science/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/pavel/Projects/Science/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


In [3]:
dataset = load_dataset("jakeazcona/short-text-multi-labeled-emotion-classification")

In [4]:
pd_dataset = dataset['train'].to_pandas()
pd_dataset.drop('Unnamed: 0', axis=1, inplace=True)
pd_dataset

,sample,emotion
0,i didnt feel humiliated,5
1,i can go from feeling so hopeless to so damned...,5
2,im grabbing a minute to post i feel greedy wrong,0
3,i am ever feeling nostalgic about the fireplac...,3
4,i am feeling grouchy,0
...,...,...
30035,Terribly sad story. I loved it.,5
30036,It's in case he suffers from amnesia and forge...,5
30037,Gotta love these kids! When they do or say the...,3
30038,Omg [NAME]! One of my favorite bb players,4


In [5]:
# Для скорости обучения возьмем только 28% датасета
_, part_dataset = train_test_split(pd_dataset, test_size=0.28, stratify=pd_dataset['emotion'], random_state=42)

In [6]:
train_df, test_df = train_test_split(part_dataset, test_size=0.2, stratify=part_dataset['emotion'], random_state=42)

In [7]:
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['emotion'], random_state=42)

In [8]:
dataset = datasets.dataset_dict.DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'validation': Dataset.from_pandas(val_df),
    'test': Dataset.from_pandas(test_df)
  })

In [9]:
def preproces_dataset(raw_dataset):
  df = pd.DataFrame(raw_dataset)
  one_hot_encoded = pd.get_dummies(df['emotion'], prefix='emotion')
  df = pd.concat([df, one_hot_encoded], axis=1)
  df.drop('emotion', axis=1, inplace=True)
  new_dataset = Dataset.from_pandas(df)
  return new_dataset

def preproces_dict_dataset(raw_dataset):
  train_dataset = raw_dataset['train']
  validation_dataset = raw_dataset['validation']
  test_dataset = raw_dataset['test']

  train_dataset = preproces_dataset(train_dataset)
  validation_dataset = preproces_dataset(validation_dataset)
  test_dataset = preproces_dataset(test_dataset)

  dataset = datasets.dataset_dict.DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset
  })
  return dataset


In [10]:
dataset = preproces_dict_dataset(dataset)

In [11]:
dataset.remove_columns('__index_level_0__')

DatasetDict({
    train: Dataset({
        features: ['sample', 'emotion_0', 'emotion_1', 'emotion_2', 'emotion_3', 'emotion_4', 'emotion_5'],
        num_rows: 5383
    })
    validation: Dataset({
        features: ['sample', 'emotion_0', 'emotion_1', 'emotion_2', 'emotion_3', 'emotion_4', 'emotion_5'],
        num_rows: 1346
    })
    test: Dataset({
        features: ['sample', 'emotion_0', 'emotion_1', 'emotion_2', 'emotion_3', 'emotion_4', 'emotion_5'],
        num_rows: 1683
    })
})

In [12]:
labels = ['emotion_0', 'emotion_1', 'emotion_2', 'emotion_3', 'emotion_4', 'emotion_5']
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [ ]:
MODEL_NAME = "albert/albert-base-v2"

In [14]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(f"{MODEL_NAME}",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert/albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

if model.config.pad_token_id == None:
    model.config.pad_token_id = model.config.eos_token_id
    model.config.pad_token = tokenizer.pad_token

In [17]:
def preprocess_data(examples):
  # take a batch of texts
  text = examples["sample"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  n_samples, sample_len = np.shape(np.array(encoding['input_ids']))
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((n_samples, len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

In [18]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Map: 100%|██████████| 1683/1683 [00:00<00:00, 10606.24 examples/s]


In [19]:
# # Alternative way to preprocess data
# def tokenize_function(example):
#     return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# # here we use datacollator, that means, that batching and paddig will be applied to the dataset during the training
# tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

In [20]:
example = encoded_dataset['train'][0]
print(example.keys())
print(tokenizer.decode(example['input_ids']))
print(example['labels'])
print([id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0])

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
[CLS] i feel like i should not be surprised at this development[SEP]<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
['emotion_4']


In [21]:
encoded_dataset.set_format("torch")

In [22]:
type(encoded_dataset['train']['input_ids'])

torch.Tensor

In [23]:
batch_size = 8
metric_name = "f1"

args = TrainingArguments(
    f"{MODEL_NAME}-transfer-trained",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name
)

In [24]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

TODO: try to use raw model to predict some label

In [25]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [26]:
date = datetime.strftime(datetime.now(), "%d.%m.%Y-%H.%M.%S")
wandb.init(
    # set the wandb project where this run will be logged
    project="nlp-multilabel-classifier",
    name=f"{MODEL_NAME}-cls-{date}",

    # # track hyperparameters and run metadata
    # config={
    # "learning_rate": 0.02,
    # "architecture": "CNN",
    # "dataset": "CIFAR-100",
    # "epochs": 10,
    # }
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: rubikpf2002 (rubikpf2002-mipt). Use `wandb login --relogin` to force relogin


In [27]:
try:
    trainer.train()
except RuntimeError:
    ...

# Закрыть сессию WandB
wandb.finish()

 37%|███▋      | 500/1346 [32:47<54:09,  3.84s/it]  

{'loss': 0.316, 'grad_norm': 7.242290496826172, 'learning_rate': 1.2570579494799406e-05, 'epoch': 0.74}


                                                    
 50%|█████     | 673/1346 [49:41<41:34,  3.71s/it]

{'eval_loss': 0.19759415090084076, 'eval_f1': 0.7824738270647538, 'eval_roc_auc': 0.8581723625557207, 'eval_accuracy': 0.7481426448736999, 'eval_runtime': 237.0223, 'eval_samples_per_second': 5.679, 'eval_steps_per_second': 0.713, 'epoch': 1.0}


 74%|███████▍  | 1000/1346 [1:12:46<23:22,  4.05s/it]

...

                                                     
100%|██████████| 1346/1346 [1:43:09<00:00,  3.90s/it]

{'eval_loss': 0.14814725518226624, 'eval_f1': 0.8441210264266564, 'eval_roc_auc': 0.8972511144130758, 'eval_accuracy': 0.8164933135215453, 'eval_runtime': 254.5132, 'eval_samples_per_second': 5.289, 'eval_steps_per_second': 0.664, 'epoch': 2.0}


100%|██████████| 1346/1346 [1:43:09<00:00,  4.60s/it]


{'train_runtime': 6189.5844, 'train_samples_per_second': 1.739, 'train_steps_per_second': 0.217, 'train_loss': 0.21337163111784482, 'epoch': 2.0}


eval/accuracy,▁█
eval/f1,▁█
eval/loss,█▁
eval/roc_auc,▁█
eval/runtime,▁█
eval/samples_per_second,█▁
eval/steps_per_second,█▁
train/epoch,▁▂▅██
train/global_step,▁▂▅██
train/grad_norm,▁█
train/learning_rate,█▁


In [28]:
# model.save_pretrained("./seved_model")

In [29]:
# text = dataset['test'][18]['sentence1'] + ' ' + dataset['test'][18]['sentence2']
# true_label = [dataset['test'][18]['label_0'], dataset['test'][18]['label_1']]
# if true_label[0]:
#   true_label = 'label_0'
# else:
#   true_label = 'label_1'
# print(text)
# encoding = tokenizer(text, return_tensors="pt")
# encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
# print(encoding)
# outputs = trainer.model(**encoding)
# logits = outputs.logits
# print(logits.shape)
# sigmoid = torch.nn.Sigmoid()
# probs = sigmoid(logits.squeeze().cpu())
# print(probs)
# predictions = np.zeros(probs.shape)
# predictions[np.where(probs >= 0.5)] = 1

# # turn predicted id's into actual label names
# predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
# print(predicted_labels)
# print(true_label)

In [30]:
# устанавливаем модель в inference режим
model.eval()

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

In [31]:
test_dataset = encoded_dataset['test'] # заметим, что здесь test_dataset уже подготовлен к инференсу см. выше

In [32]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(test_dataset, batch_size=16)

In [33]:
# for batch in test_dataloader:
#     # print(batch)
#     true_batch_labels = batch['labels'].cpu().numpy()
#     print(true_batch_labels)
#     indices = [int(np.nonzero(row == 1)[0]) for row in true_batch_labels]
#     print(indices)
#     break

In [34]:
from sklearn.metrics import classification_report

In [35]:
predictions = []
true_labels = []

for batch in test_dataloader:
  # Move batch data to the same device as the model
  batch = {k: v.to(model.device) for k, v in batch.items()}

  with torch.no_grad():
    outputs = model(**batch)

  logits = outputs.logits
  preds = np.argmax(logits.cpu().numpy(), axis=1)
  predictions.extend(preds)

  true_batch_labels = batch['labels'].cpu().numpy()
  indices = np.array([int(np.nonzero(row == 1)[0]) for row in true_batch_labels])
  true_labels.extend(indices)  # Добавляем истинные метки для оценки

# 5. Оценка качества
f1_micro_average = f1_score(y_true=true_labels, y_pred=predictions, average='micro')
# roc_auc = roc_auc_score(true_labels, predictions, average = 'micro')
accuracy = accuracy_score(true_labels, predictions)
# return as dictionary
metrics = {'f1': f1_micro_average,
            # 'roc_auc': roc_auc,
            'accuracy': accuracy}
report = classification_report(true_labels, predictions)

print(f"Accuracy: {accuracy}")
print(metrics)
print(report)

Accuracy: 0.839572192513369
{'f1': 0.839572192513369, 'accuracy': 0.839572192513369}
              precision    recall  f1-score   support

           0       0.83      0.80      0.82       262
           1       0.78      0.74      0.76       175
           2       0.89      0.85      0.87       479
           3       0.81      0.82      0.81       236
           4       0.71      0.76      0.73       115
           5       0.86      0.93      0.90       416

    accuracy                           0.84      1683
   macro avg       0.81      0.82      0.81      1683
weighted avg       0.84      0.84      0.84      1683

